In [3]:
# install libraries if needed
!pip install mysql-connector-python
!pip install pandas
!pip install numpy
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 30.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
# Some functions for connecting to the database - courtesy of Bill
import mysql.connector

def create_conn():
    config = {
        'user': 'teamDelta',
        'password': 'teamDelta@1234',
        'host': 'watstreet-data-cluster-instance-1.cymafucs9kp7.us-east-2.rds.amazonaws.com',
        'database': 'watstreetData',
        'raise_on_warnings': True
    }

    try:
        cnx = mysql.connector.connect(**config)
        return cnx
    except mysql.connector.Error as err:
        print(err)

def database_summary(connection):
    cursor = connection.cursor()
    
    # get list of all tables
    cursor.execute("SHOW TABLES")
    tables = cursor.fetchall()

    for table in tables:
        print(f"\nTable: {table[0]}")
        
        # get table structure
        cursor.execute(f"DESCRIBE {table[0]}")
        structure = cursor.fetchall()
        print("Structure:")
        for column in structure:
            print(column)

In [3]:
# create a db connection
conn = create_conn()

In [4]:
# Summary of data
database_summary(conn)


Table: cryptoTrades
Structure:
('uid', b'int', 'NO', 'PRI', None, '')
('time', b'double', 'YES', '', None, '')
('trade_id', b'text', 'YES', '', None, '')
('price', b'double', 'YES', '', None, '')
('amount', b'double', 'YES', '', None, '')
('exchange', b'text', 'YES', '', None, '')
('instrmnt', b'text', 'YES', '', None, '')
('vol_dllr', b'double', 'YES', '', None, '')
('sell2', b'double', 'YES', '', None, '')
('Class', b'text', 'YES', '', None, '')
('Asset', b'text', 'YES', '', None, '')
('datetime', b'text', 'YES', '', None, '')
('year', b'int', 'YES', '', None, '')
('hour', b'int', 'YES', '', None, '')


The data exists until August 15, 2022.

In [5]:
# tickers = [
#     'ssv-usdt','fxs-usdt','perl-usdt','rdnt-usdt','gns-usdt','near-usdt','op-usdt','rpl-usdt','ldo-usdt','tru-usdt','arb-usdt'
# ]

tickers = [
    'btc-usdt', 'eth-usdt', 'xrp-usdt', 'doge-usdt', 'bch-usdt', 'tru-usdt', 'aave-usdt'
]

In [6]:
cursor = conn.cursor()
# cursor.execute(
#     f"""
#     SELECT datetime, time, price, amount, exchange, instrmnt, vol_dllr, sell2, asset 
#     FROM cryptoTrades WHERE time > {start_date} and time < {end_date}
#     and (exchange = 'bnce' or exchange = 'polo')
#     """
# )
cursor.execute(
    f"""
    SELECT datetime, time, price, amount, exchange, instrmnt, vol_dllr, sell2, asset 
    FROM cryptoTrades 
    WHERE (exchange = 'bnce' OR exchange = 'polo') AND instrmnt IN ({', '.join(['%s'] * len(tickers))})
    """,
    params = tickers
)

In [7]:
x = cursor.fetchall()

In [8]:
len(x)

7551118

In [9]:
df = pd.DataFrame(x)
columns = ["datetime", "time", "price", "amount", "exchange", "instrmnt", "vol_dllr", "sell2", "asset"]
df.columns = columns
df.set_index("datetime", inplace=True)
df.index = pd.to_datetime(df.index)
df.sort_index(inplace=True)
df

,time,price,amount,exchange,instrmnt,vol_dllr,sell2,asset
datetime,,,,,,,,
2022-08-15 00:00:00.000999936,1.660522e+12,1935.32,0.03080,bnce,eth-usdt,59.607856,0.0,eth
2022-08-15 00:00:00.000999936,1.660522e+12,1935.31,0.00950,bnce,eth-usdt,18.385445,1.0,eth
2022-08-15 00:00:00.001999872,1.660522e+12,24305.25,0.00063,bnce,btc-usdt,15.312308,1.0,btc
2022-08-15 00:00:00.004000000,1.660522e+12,24307.02,0.00073,bnce,btc-usdt,17.744125,0.0,btc
2022-08-15 00:00:00.007000064,1.660522e+12,1935.31,0.01850,bnce,eth-usdt,35.803235,1.0,eth
...,...,...,...,...,...,...,...,...
2022-08-15 23:59:59.993999872,1.660608e+12,24094.82,0.00057,bnce,btc-usdt,13.734047,0.0,btc
2022-08-15 23:59:59.996000000,1.660608e+12,1899.06,0.04520,bnce,eth-usdt,85.837512,0.0,eth
2022-08-15 23:59:59.996000000,1.660608e+12,1899.06,0.04340,bnce,eth-usdt,82.419204,0.0,eth


In [10]:
df.instrmnt.unique()

array(['eth-usdt', 'btc-usdt', 'xrp-usdt', 'bch-usdt', 'doge-usdt',
       'tru-usdt', 'aave-usdt'], dtype=object)

In [11]:
df.exchange.unique()

array(['bnce', 'polo'], dtype=object)

In [12]:
df.to_parquet("Binance-Poloniex-Hugh.parquet")

In [49]:
df["flow"] = df.sell2.replace(0, -1) * -1
df["flow"] = df.flow * df.amount

In [20]:
df.head()

,time,price,amount,exchange,instrmnt,vol_dllr,sell2,asset
datetime,,,,,,,,
2022-08-15 00:00:00.001999872,1.660522e+12,24305.25,0.00063,bnce,btc-usdt,15.312308,1.0,btc
2022-08-15 00:00:00.004000000,1.660522e+12,24307.02,0.00073,bnce,btc-usdt,17.744125,0.0,btc
2022-08-15 00:00:00.008000000,1.660522e+12,24305.25,0.00090,bnce,btc-usdt,21.874725,1.0,btc
2022-08-15 00:00:00.008000000,1.660522e+12,24307.02,0.00117,bnce,btc-usdt,28.439213,0.0,btc
2022-08-15 00:00:00.008000000,1.660522e+12,24307.02,0.00092,bnce,btc-usdt,22.362458,0.0,btc
